In [ ]:
import pandas as pd
import numpy as np

import os

In [ ]:
# path to project: Edit here to point to your Drive
drive = '/content/drive/MyDrive/'
project_dir = 'Colab Notebooks/IronHacks2024_Data_Analytics/DA_Week_8-9_Final_Project'
COLAB_PROJECT_PATH = drive+project_dir+'/Final_Project/'

In [ ]:
# Mount Google Drive
using_colab = False
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)
    using_colab = True

if using_colab and os.path.exists(COLAB_PROJECT_PATH):
    print('found colab path; redirecting to main project directory')
    os.chdir(COLAB_PROJECT_PATH)

Mounted at /content/drive
found colab path; redirecting to main project directory


### Import recommendation data

In [ ]:
vg_rec_data = pd.read_csv('./Recommendation_Data/vg_cluster_data.csv')

### Search Algorithim


In [ ]:
vg_rec_data

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,User_Score,Publisher_Profile,Cluster
0,32,Wii Sports,Wii,2006,Sports,Nintendo,41.36,29.02,3.77,8.51,7.7,0.0,High Profile,4
1,52,Mario Kart 8 Deluxe,NS,2017,Racing,Nintendo,5.05,4.98,2.11,0.91,9.3,0.0,High Profile,18
2,81,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,10.0,8.2,High Profile,12
3,87,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.91,12.92,3.80,3.35,8.2,9.1,High Profile,18
4,94,Tetris,"GB, NES",1989,Puzzle,Nintendo,26.17,2.95,6.03,0.69,0.0,0.0,High Profile,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12022,20793,1/2 Summer +,PSP,2013,Misc,Kaga Create,0.00,0.00,0.01,0.00,0.0,0.0,Low Profile,0
12023,20797,Pippa Funnell 2: Farm Adventures,DS,2007,Simulation,Ubisoft,0.00,0.01,0.00,0.00,0.0,0.0,High Profile,20
12024,20798,Irotoridori no Sekai: World's End Re-Birth,PSV,2015,Action,HuneX,0.00,0.00,0.01,0.00,0.0,0.0,Low Profile,22
12025,20799,PDC World Championship Darts 2009,Wii,2009,Sports,Oxygen Interactive,0.00,0.01,0.00,0.00,0.0,0.0,Low Profile,24


In [ ]:
def search_game(game_name:str):
    """
    Search for game in dataframe, get all potential entries

    Arguments: name of game to search for
    Returns: pd.DataFrame - filtered
    """

    # Convert game_name to lowercase for case-insensitive matching
    game_name_lower = game_name.lower().strip()

    # Filter dataframe for rows where 'Game Name' contains game_name_lower
    vg_rec_data['Game_Name'] = vg_rec_data['Name'].str.lower().str.strip()

    filtered_results = vg_rec_data[vg_rec_data['Game_Name'].str.contains(game_name_lower)].reset_index()

    vg_rec_data.drop(['Game_Name'], axis=1, inplace=True)
    filtered_results = filtered_results.drop(['Game_Name'], axis=1)

    return filtered_results[['Rank', 'Name', 'Platform', 'Genre', 'Cluster']]

In [ ]:
search_game('Mega Man X')

,Rank,Name,Platform,Genre,Cluster
0,3158,Mega Man X,SNES,Platform,21
1,3467,Mega Man X4,"PS, SAT",Platform,21
2,4570,Mega Man X5,PS,Platform,21
3,5406,Mega Man X7,PS2,Platform,21
4,5596,Mega Man X2,SNES,Platform,21
5,6891,Mega Man X6,PS,Platform,21
6,7649,Mega Man X3,SNES,Platform,21
7,8557,Mega Man X8,PS2,Platform,21
8,9528,Mega Man X Collection,"PS2, GC",Misc,19
9,11129,Mega Man X: Command Mission,"PS2, GC",Role-Playing,5


### Recommendation algorithm

In [ ]:
def get_cluster(cluster_group):
    cluster_filter = vg_rec_data['Cluster'] == cluster_group
    return vg_rec_data[cluster_filter]

def get_recommendations(game_data, num_results:int=10, page:int=1):
    """
    Grabs results for game via its cluster grouping and sorted by critic score
    then User Score in descending order, then by rank in ascending order.

    Arguments:
        game_data (pd.Series) : Data for game to use to get recommnedations
        num_results - Number of results to output
        page - Representes chunk of cluster to data to get data from

    Returns:
        pd.DataFrame - The recommended data at page # with num results
    """
    # print(game_data['Cluster'])
    games_in_cluster = get_cluster(game_data['Cluster'])

    # Make sure to remove game so it doesn't show up in results
    remove_game = games_in_cluster['Name'] != game_data['Name']
    games_in_cluster = games_in_cluster[remove_game]

    # Sort By Critic score, then in User Score, both in descending order
    # Then sort by Rank in Ascending Order
    sort_dict = {'by':['Critic_Score', 'User_Score','Rank'], 'ascending':[False, False, True]}
    games_in_cluster = games_in_cluster.sort_values(**sort_dict).reset_index()

    # Get results for page # with num_results
    start_res, end_res = ((page-1) * num_results, page * num_results)
    page_results = games_in_cluster.iloc[start_res: end_res]

    return page_results

In [ ]:
def Get_Game_for_Rec():
    """
    Prompts User to input a game, and get the game data for recommendation system

    Returns: pd.series - The game entry chosen by User
    """

    # Prompt User for game they like for game recommendations
    while True:
        search_for_game = input("Input a game you like: ")
        search_results = search_game(search_for_game)
        search_results.index += 1

        if len(search_results) > 0:
            print('\n'+'='*30+'\n', end='')
            break
        print('Game Not Found in DataBase: Please try inputting again.', \
               end='\n'+'-'*30+'\n')

    game_for_rec = search_results.iloc[0]
    print(search_results.drop('Cluster',axis=1), end='\n'+'='*30+'\n')

    # if more than one result, prompt user to pick one specific entry
    more_than_one = len(search_results) > 1
    while more_than_one:
        try:
            print("Found Multiple Results: Please pick one from the list using the number choices")
            game_index = int(input("Which Game do you want to input: "))
            game_for_rec = search_results.iloc[game_index-1]
            more_than_one = False
        except IndexError:
            print('-'*50)
            print(f"Index {game_index} is not a valid choice. Try again.")
            print('-'*50)
        except ValueError:
            print('-'*50)
            print('Not valid entry of number: try again.')
            print('-'*50)

    print('-'*50)
    print(game_for_rec['Name'], end='\n'+'-'*30+'\n')
    return game_for_rec

In [ ]:
def video_game_recommendations():
    """
    Main Function for video game recommendation system:
    Grabs a search term, asks user to choose specific game
    if found more than one result, and grab recommendations
    sorted by scores (descending), with pagination.
    """

    game_entry = Get_Game_for_Rec()

    print('Each page produces 10 results')
    while True:
        try:
            page_num = int(input('How many pages of games would you like recommended? : '))
            break
        except ValueError:
            print('Not a number: try again.')

    total_results = 10 * page_num
    games_recommendations = get_recommendations(game_entry, total_results)
    display(games_recommendations[['Name','Platform','Genre','Publisher']])

In [ ]:
video_game_recommendations()

Input a game you like: Super Smash Bros

   Rank                         Name Platform     Genre
1    96   Super Smash Bros. Ultimate       NS  Fighting
2   310      Super Smash Bros. Brawl      Wii  Fighting
3   456    Super Smash Bros. for 3DS      3DS  Fighting
4   554      Super Smash Bros. Melee       GC  Fighting
5   710            Super Smash Bros.      N64  Fighting
6   734  Super Smash Bros. for Wii U     WiiU  Fighting
Found Multiple Results: Please pick one from the list using the number choices
Which Game do you want to input: 2
--------------------------------------------------
Super Smash Bros. Brawl
------------------------------
Each page produces 10 results
How many pages of games would you like recommended? : 4


,Name,Platform,Genre,Publisher
0,Super Smash Bros. Ultimate,NS,Fighting,Nintendo
1,Virtua Fighter 4,PS2,Fighting,Sega
2,Super Smash Bros. Melee,GC,Fighting,Nintendo
3,Super Smash Bros. for Wii U,WiiU,Fighting,Nintendo
4,Virtua Fighter 5 Online,X360,Fighting,Sega
5,WWF No Mercy,N64,Fighting,THQ
6,Virtua Fighter,SAT,Fighting,Sega
7,Virtua Fighter 4: Evolution,PS2,Fighting,Sega
8,WWF SmackDown! 2: Know Your Role,PS,Fighting,THQ
9,Punch-Out!!,Wii,Fighting,Nintendo


### That's the recommendation system
In Future, I might want to use recommendation engine techniques such as collaborative filtering and Sparse Matrix Transformations to get mo